## Data Ingestion Notebook

In [1]:
import os 
os.chdir("../")

In [2]:
pwd

'/home/jvdboss/workspace/ML_DL/image_classification/Lung_Xray_Classifier'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [4]:
from xray.constants import *
from xray.utils import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [6]:
import os
import urllib.request as request
from zipfile import ZipFile

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )

    def _get_updated_list_of_files(self, list_of_files):
        return [f for f in list_of_files if f.endswith(".jpeg") and ("NORMAL" in f or "PNEUMONIA" in f)]

    def _preprocess(self, zf: ZipFile, f: str, working_dir: str):
        target_filepath = os.path.join(working_dir, f)
        if not os.path.exists(target_filepath):
            zf.extract(f, working_dir)
        
        if os.path.getsize(target_filepath) == 0:
            os.remove(target_filepath)
            
    def unzip_and_clean(self):
        
        with ZipFile(file=self.config.local_data_file, mode="r") as zf:
            print(self.config.local_data_file)
            list_of_files = zf.namelist()
            updated_list_of_files = self._get_updated_list_of_files(list_of_files)
            print(updated_list_of_files)
            for f in updated_list_of_files:
                self._preprocess(zf, f, self.config.unzip_dir)

In [7]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzip_and_clean()
except Exception as e:
    raise e

[2022-10-12 12:22:24,505: INFO: common]: yaml file: configs/config.yaml loaded successfully
[2022-10-12 12:22:24,508: INFO: common]: yaml file: params.yaml loaded successfully
[2022-10-12 12:22:24,509: INFO: common]: created directory at: artifacts
[2022-10-12 12:22:24,509: INFO: common]: created directory at: artifacts/data_ingestion
artifacts/data_ingestion/chest_xray.zip
['chest_xray/NORMAL/NORMAL-1003233-0001.jpeg', 'chest_xray/NORMAL/NORMAL-1012843-0001.jpeg', 'chest_xray/NORMAL/NORMAL-1014768-0001.jpeg', 'chest_xray/NORMAL/NORMAL-1023731-0001.jpeg', 'chest_xray/NORMAL/NORMAL-1029510-0001.jpeg', 'chest_xray/NORMAL/NORMAL-1031320-0001.jpeg', 'chest_xray/NORMAL/NORMAL-1034239-0001.jpeg', 'chest_xray/NORMAL/NORMAL-1041364-0001.jpeg', 'chest_xray/NORMAL/NORMAL-1044645-0001.jpeg', 'chest_xray/NORMAL/NORMAL-132181-0001.jpeg', 'chest_xray/NORMAL/NORMAL-133685-0001.jpeg', 'chest_xray/NORMAL/NORMAL-137300-0001.jpeg', 'chest_xray/NORMAL/NORMAL-141619-0001.jpeg', 'chest_xray/NORMAL/NORMAL-14